In [ ]:
"""

# select 1000 unlabeled data to label 

selected_unlabeld_data = select_unlabeled_data(all_unlabeled_data, strategy="random", num=1000)
selected_unlabeld_data = select_unlabeled_data(all_unlabeled_data, strategy="uncertainty_sampling", num=1000)
selected_unlabeld_data = select_unlabeled_data(all_unlabeled_data, strategy="optimal_subset", optimal_subset=optimal_subset)

# label these data (in this case, we just get their labels from the HF dataset, because they are already labeled and we were only pretending that they were unlabeled)

selected_labeled_data = label_data(selected_unlabeld_data, hf_data)

# train the model with these newly labeled data and plot them out 

"""